# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

In [1]:
# %cd /mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT
# !mkdir -p models
# %cd ./models
# !mkdir -p sentiments
# %cd ./sentiments
# !wget https://prosus-public.s3-eu-west-1.amazonaws.com/finbert/finbert-sentiment/pytorch_model.bin

## Modules 

In [2]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

In [3]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [4]:
# lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
# cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
# cl_data_path = project_dir/'data'/'sentiment_data'

print(project_dir)
lm_path = project_dir/"models"/"language"
cl_path = project_dir/"models"/"sentiment"
cl_data_path = project_dir/"data"/"sentiment_data"

/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT


###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [5]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

Some weights of the model checkpoint at /mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/models/language were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequence

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [6]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [7]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

04/18/2021 20:24:10 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


## Fine-tune the model

In [8]:
# Get the training examples
train_data = finbert.get_data('train')

In [9]:
model = finbert.create_the_model()

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [ ]:
# This is for fine-tuning a subset of the model.

freeze = 6

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [10]:
trained_model = finbert.train(train_examples = train_data, model = model)

04/18/2021 20:24:21 - INFO - finbert.utils -   *** Example ***
04/18/2021 20:24:21 - INFO - finbert.utils -   guid: train-1
04/18/2021 20:24:21 - INFO - finbert.utils -   tokens: [CLS] we can capital ##ize on our experience from a number of similar bridge projects in the nordic countries . [SEP]
04/18/2021 20:24:21 - INFO - finbert.utils -   input_ids: 101 2057 2064 3007 4697 2006 2256 3325 2013 1037 2193 1997 2714 2958 3934 1999 1996 13649 3032 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:24:21 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:24:21 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:24:21 - INFO - finbert.utils -   label: positive (id = 0)
04/18/2021 20:24:22 - INFO - finbert.finbert -   ***** Loading data *****
04/18/2021 20:24:22 - I

Iteration:   0%|          | 0/91 [00:00<?, ?it/s]

04/18/2021 20:24:57 - INFO - finbert.utils -   *** Example ***
04/18/2021 20:24:57 - INFO - finbert.utils -   guid: validation-1
04/18/2021 20:24:57 - INFO - finbert.utils -   tokens: [CLS] in january - september 2007 , finn ##lines ' net sales rose to eu ##r 505 . 4 mn from eu ##r 47 ##3 . 5 mn in the corresponding period in 2006 . [SEP]
04/18/2021 20:24:57 - INFO - finbert.utils -   input_ids: 101 1999 2254 1011 2244 2289 1010 9303 12735 1005 5658 4341 3123 2000 7327 2099 28952 1012 1018 24098 2013 7327 2099 4700 2509 1012 1019 24098 1999 1996 7978 2558 1999 2294 1012 102 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:24:57 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:24:57 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:24:57 - INFO - finbert.utils -   label: positive (id = 0)
04/18

Validating:   0%|          | 0/31 [00:00<?, ?it/s]

Validation losses: [0.7034077317483963]
No best model found


...

Iteration:   0%|          | 0/91 [00:00<?, ?it/s]

04/18/2021 20:29:18 - INFO - finbert.utils -   *** Example ***
04/18/2021 20:29:18 - INFO - finbert.utils -   guid: validation-1
04/18/2021 20:29:18 - INFO - finbert.utils -   tokens: [CLS] in january - september 2007 , finn ##lines ' net sales rose to eu ##r 505 . 4 mn from eu ##r 47 ##3 . 5 mn in the corresponding period in 2006 . [SEP]
04/18/2021 20:29:18 - INFO - finbert.utils -   input_ids: 101 1999 2254 1011 2244 2289 1010 9303 12735 1005 5658 4341 3123 2000 7327 2099 28952 1012 1018 24098 2013 7327 2099 4700 2509 1012 1019 24098 1999 1996 7978 2558 1999 2294 1012 102 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:29:18 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:29:18 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 20:29:18 - INFO - finbert.utils -   label: positive (id = 0)
04/18

Validating:   0%|          | 0/31 [00:00<?, ?it/s]

Epoch: 100%|██████████| 4/4 [05:06<00:00, 76.70s/it]

Validation losses: [0.7034077317483963, 0.4108896553516388, 0.39502903314367416, 0.39613443877427806]


## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [13]:
test_data = finbert.get_data('test')

In [14]:
results = finbert.evaluate(examples=test_data, model=trained_model)

04/18/2021 19:23:30 - INFO - finbert.utils -   *** Example ***
04/18/2021 19:23:30 - INFO - finbert.utils -   guid: test-1
04/18/2021 19:23:30 - INFO - finbert.utils -   tokens: [CLS] finnish it solutions provider affect ##o o ##y ##j said today won a frame contract , valued at some eu ##r ##2 ##m , to implement the next phase of its insurance application for south african mutual & federal insurance company limited m & f . [SEP]
04/18/2021 19:23:30 - INFO - finbert.utils -   input_ids: 101 6983 2009 7300 10802 7461 2080 1051 2100 3501 2056 2651 2180 1037 4853 3206 1010 11126 2012 2070 7327 2099 2475 2213 1010 2000 10408 1996 2279 4403 1997 2049 5427 4646 2005 2148 3060 8203 1004 2976 5427 2194 3132 1049 1004 1042 1012 102
04/18/2021 19:23:30 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
04/18/2021 19:23:30 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

Testing:   0%|          | 0/31 [00:00<?, ?it/s]

### Prepare the classification report

In [15]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [16]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [17]:
report(results,cols=['labels','prediction','predictions'])

Loss:0.36
Accuracy:0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.85      0.80       272
           1       0.75      0.89      0.81       120
           2       0.92      0.84      0.88       575

    accuracy                           0.85       967
   macro avg       0.81      0.86      0.83       967
weighted avg       0.86      0.85      0.85       967



### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [18]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [21]:
#cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/max/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
result = predict(text,model)

04/18/2021 19:24:55 - INFO - finbert.utils -   *** Example ***
04/18/2021 19:24:55 - INFO - finbert.utils -   guid: 0
04/18/2021 19:24:55 - INFO - finbert.utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
04/18/2021 19:24:55 - INFO - finbert.utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 19:24:55 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 19:24:55 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [24]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.018325487, 0.9405822, 0.041092336]",negative,-0.922257,0.051746
1,The news rapidly infected financial markets.,"[0.07987844, 0.62806284, 0.29205877]",negative,-0.548184,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.0153823765, 0.9639771, 0.020640507]",negative,-0.948595,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.03195808, 0.9523825, 0.015659302]",negative,-0.920424,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.012949453, 0.95318514, 0.033865355]",negative,-0.940236,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.035394415, 0.8892764, 0.07532924]",negative,-0.853882,0.000000


In [25]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.86.


Here is another example

In [26]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [27]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

04/18/2021 19:25:45 - INFO - finbert.utils -   *** Example ***
04/18/2021 19:25:45 - INFO - finbert.utils -   guid: 0
04/18/2021 19:25:45 - INFO - finbert.utils -   tokens: [CLS] shares in the spin - off of south african e - commerce group nas ##pers surged more than 25 % in the first minutes of their market debut in amsterdam on wednesday . [SEP]
04/18/2021 19:25:45 - INFO - finbert.utils -   input_ids: 101 6661 1999 1996 6714 1011 2125 1997 2148 3060 1041 1011 6236 2177 17235 7347 18852 2062 2084 2423 1003 1999 1996 2034 2781 1997 2037 3006 2834 1999 7598 2006 9317 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 19:25:45 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/18/2021 19:25:45 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [28]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.9401025, 0.037251446, 0.022646042]",positive,0.902851,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.020288479, 0.015229413, 0.96448207]",neutral,0.005059,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.14337526, 0.012087587, 0.8445372]",neutral,0.131288,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.9351332, 0.025614208, 0.039252598]",positive,0.909519,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.71728885, 0.022203213, 0.2605079]",positive,0.695086,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.093661755, 0.014246225, 0.89209205]",neutral,0.079416,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.7191518, 0.018251665, 0.26259655]",positive,0.700900,0.000000


In [29]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.49.
